In [57]:
import gradio as gr
import pandas as pd
import mlflow
import numpy as np
import shap
import mlflow.pyfunc
import sklearn
import pickle
import shap
import xgboost as xgb

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

In [47]:
model = load_model('rent-wue-selected-features-linear-reg')
print(model)
model_xgb = load_model('rent_wue_selected_features_xgb-aug-0')

print("--------------")
print(model_xgb)

2023/07/13 22:59:03 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: fe6a7fd085f04fc88093bdb1c0208f94



2023/07/13 22:59:13 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


--------------
mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: fb73c615ce3e441294abf21df52d5479



In [44]:
def load_model(model_name):
    model_version = 1
    model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/production")
    return model

In [107]:
def trigger_actions(
    feature_squrmeter,
    feature_zip,
    feature_rooms,
    features_altbau,
    feature_balkon,
    feature_ba,
    feature_dachgeschoss,
    feature_einbaukueche,
    feature_neubau,
    feature_parkett,
    feature_stellplatz,
    feature_badwc_getrennt,
    feature_personenaufzug,
    feature_garten,
    feature_garage,
    feature_renoviert,
    feature_terrasse,
    feature_wanne,
    feature_zentralheizung,
    feature_abstellraum,
    feature_fernwaerme,
    feature_fussbodenheitzung,
    feature_gartenmitbenutzung,
    feature_kellerabteil,
    erklärung
):
    data_list = (
        [feature_squrmeter]
        + [feature_rooms]
        + [features_altbau]
        + [feature_balkon]
        + [feature_ba]
        + [feature_dachgeschoss]
        + [feature_einbaukueche]
        + [feature_neubau]
        + [feature_parkett]
        + [feature_stellplatz]
        + [feature_badwc_getrennt]
        + [feature_personenaufzug]
        + [feature_garten]
        + [feature_garage]
        + [feature_renoviert]
        + [feature_terrasse]
        + [feature_wanne]
        + [feature_zentralheizung]
        + [feature_abstellraum]
        + [feature_fernwaerme]
        + [feature_fussbodenheitzung]
        + [feature_gartenmitbenutzung]
        + [feature_kellerabteil]
        + [int(0)] * 23
    )
    data = pd.DataFrame()
    data = pd.DataFrame(
        columns=[
            "LivingSpace",
            "Rooms",
            "altbau_(bis_1945)",
            "balkon",
            "barriefrei",
            "dachgeschoss",
            "einbaukueche",
            "neubau",
            "parkett",
            "stellplatz",
            "bad/wc_getrennt",
            "personenaufzug",
            "garten",
            "garage",
            "renoviert",
            "terrasse",
            "wanne",
            "zentralheizung",
            "abstellraum",
            "ferne",
            "fussbodenheizung",
            "gartennutzung",
            "kelleranteil",
            "ZipCode_97070",
            "ZipCode_97072",
            "ZipCode_97074",
            "ZipCode_97076",
            "ZipCode_97078",
            "ZipCode_97080",
            "ZipCode_97082",
            "ZipCode_97084",
            "ZipCode_97204",
            "ZipCode_97209",
            "ZipCode_97218",
            "ZipCode_97222",
            "ZipCode_97228",
            "ZipCode_97234",
            "ZipCode_97236",
            "ZipCode_97246",
            "ZipCode_97249",
            "ZipCode_97250",
            "ZipCode_97261",
            "ZipCode_97270",
            "ZipCode_97288",
            "ZipCode_97297",
            "ZipCode_97299",
        ]
    )

    data.loc[len(data)] = data_list
    for index, row in data.iterrows():
        for columns in data.columns:
            if feature_zip[:5] in columns:
                data.loc[index, columns] = int(1)

    data = data.replace(False, 0)
    data = data.replace(True, 1)
    data = data.astype({"ZipCode_97070": np.int32})
    data = data.astype({"ZipCode_97072": np.int32})
    data = data.astype({"ZipCode_97074": np.int32})
    data = data.astype({"ZipCode_97076": np.int32})
    data = data.astype({"ZipCode_97078": np.int32})
    data = data.astype({"ZipCode_97080": np.int32})
    data = data.astype({"ZipCode_97082": np.int32})
    data = data.astype({"ZipCode_97084": np.int32})
    data = data.astype({"ZipCode_97204": np.int32})
    data = data.astype({"ZipCode_97209": np.int32})
    data = data.astype({"ZipCode_97218": np.int32})
    data = data.astype({"ZipCode_97222": np.int32})
    data = data.astype({"ZipCode_97228": np.int32})
    data = data.astype({"ZipCode_97234": np.int32})
    data = data.astype({"ZipCode_97236": np.int32})
    data = data.astype({"ZipCode_97246": np.int32})
    data = data.astype({"ZipCode_97249": np.int32})
    data = data.astype({"ZipCode_97250": np.int32})
    data = data.astype({"ZipCode_97261": np.int32})
    data = data.astype({"ZipCode_97270": np.int32})
    data = data.astype({"ZipCode_97288": np.int32})
    data = data.astype({"ZipCode_97297": np.int32})
    data = data.astype({"ZipCode_97299": np.int32})

    html_response = ""
    fig_waterfall = None
    model_xgb = None
    if (not erklärung): 
        data_model = data
        preds = model.predict(data_model)
        preds = int(preds)
        print(preds)
        print(type(preds))
        html_response = f"""
            <html>
                <style>
                    body {{
                        font-family: Arial, sans-serif;
                        margin-top: 200px;
                    }}
                    
                    h1 {{
                        color: #333;
                        font-size: 24px;
                        text-align: center;
                    }}
                    
                    p {{
                        color: #777;
                        font-size: 22px;
                        text-align: center;
                    }}
                </style>
            </head>
            <body>
                <h1>Der vorgeschlagene Preis für die Immobilie beträgt:</h1>
                <p>{preds} €</p>
            </body>
            </html>
        """
    elif(erklärung):
        model_xgb = xgb.XGBRegressor()
        print("xgb loaded")
        model_xgb.load_model(r'model.xgb')
        print(model_xgb)
        # print the columns of data
        print(f"Data cols: {data.columns}")
        # prin the amount of columns
        print(f"Data cols: {len(data.columns)}")
        preds = model_xgb.predict(data)
        print("predictions shape")
        print(preds.shape)
        print("predictions done")
        html_response = f"""
            <html>
                <style>
                    body {{
                        font-family: Arial, sans-serif;
                        margin-top: 200px;
                    }}
                    
                    h1 {{
                        color: #333;
                        font-size: 24px;
                        text-align: center;
                    }}
                    
                    p {{
                        color: #777;
                        font-size: 22px;
                        text-align: center;
                    }}
                </style>
            </head>
            <body>
                <h1>Der vorgeschlagene Preis für die Immobilie beträgt:</h1>
                <p>{preds} €</p>
            </body>
            </html>
        """

        print("shap start")
        explainer = shap.Explainer(model_xgb)
        print("explainer loaded")
        print(preds)
  


        # reshape the predictions to the correct shape
        preds = preds.reshape(1, -1)
        shap_values = explainer(preds)
        print("shap values loaded")

        fig_waterfall =  shap.plots.waterfall(shap_values[0])
        print("waterfall done")

    return html_response, fig_waterfall

In [108]:
with gr.Blocks() as demo:
    gr.Markdown(
    """
    # Geben Sie hier an, welche Merkmale Ihre Immobilie besitzt 🏠:
    """
    )
    with gr.Row():
        feature_zip = gr.Dropdown(
            label="Postleitzahl",
            choices=[
                "97070 Würzburg-Altstadt",
                "97072 Würzburg-Sanderau",
                "97074 Würzburg-Frauenland",
                "97076 Würzburg-Lengfeld",
                "97078 Würzburg-Lindleinsmühle",
                "97080 Würzburg-Dürrbachtal",
                "97082 Würzburg-Steinbachtal",
                "97084 Würzburg-Heuchelhof",
                "97204 Höchberg",
                "97209 Veitshöchheim",
                "97218 Gerbrunn",
                "97222 Rimpar",
                "97228 Rottendorf",
                "97234 Reichenberg",
                "97236 Randersacker",
                "97246 Eibelstadt",
                "97249 Eisingen",
                "97250 Erlabrunn",
                "97261 Güntersleben",
                "97270 Kist",
                "97288 Theilheim",
                "97297 Waldbüttelbrunn",
                "97299 Zell am Main",
            ],
            value="97070 Würzburg-Altstadt",
        )
    with gr.Row():
        feature_squrmeter = gr.Number(label="Wohnfläche [qm]", value=79)
    with gr.Row():
        feature_rooms = gr.Number(label="Anzahl der Zimmer", value=3)
    with gr.Row():
        features_altbau = gr.Checkbox(label="Altbau (bis 1945)")
        feature_balkon = gr.Checkbox(label="Balkon")
        feature_ba = gr.Checkbox(label="Barrierefrei")
        feature_dachgeschoss = gr.Checkbox(label="Dachgeschoss")
        feature_einbaukueche = gr.Checkbox(label="Einbauküche")
        feature_neubau = gr.Checkbox(label="Neubau")
        feature_parkett = gr.Checkbox(label="Parkett")
        feature_stellplatz = gr.Checkbox(label="Stellplatz")
        feature_badwc_getrennt = gr.Checkbox(label="Bad/WC getrennt")
        feature_personenaufzug = gr.Checkbox(label="Personenaufzug")
        feature_garten = gr.Checkbox(label="Garten")
        feature_garage = gr.Checkbox(label="Garage")
        feature_renoviert = gr.Checkbox(label="Renoviert")
        feature_terrasse = gr.Checkbox(label="Terrasse")
        feature_wanne = gr.Checkbox(label="Wanne")
        feature_zentralheizung = gr.Checkbox(label="Zentralheizung")
        feature_abstellraum = gr.Checkbox(label="Abstellraum")
        feature_fernwaerme = gr.Checkbox(label="Fernwärme")
        feature_fussbodenheitzung = gr.Checkbox(label="Fußbodenheizung")
        feature_gartenmitbenutzung = gr.Checkbox(label="Gartenmitbenutzung")
        feature_kellerabteil = gr.Checkbox(label="Kellerabteil")

    with gr.Accordion("erweiterte Einstellungen"): 
        erklärung = gr.Checkbox(label="Modelvorhersage begründen (mit SHAP, xgb als modell verwendet)")

    btn = gr.Button("Berechne deinen Preis")

    output_html = gr.HTML(label="Dein Preis")
    waterfall_plot = gr.Plot(label="SHAP Waterfall Plot", visible=True)
    btn.click(
        fn=trigger_actions,
        inputs=[
            feature_squrmeter,
            feature_zip,
            feature_rooms,
            features_altbau,
            feature_balkon,
            feature_ba,
            feature_dachgeschoss,
            feature_einbaukueche,
            feature_neubau,
            feature_parkett,
            feature_stellplatz,
            feature_badwc_getrennt,
            feature_personenaufzug,
            feature_garten,
            feature_garage,
            feature_renoviert,
            feature_terrasse,
            feature_wanne,
            feature_zentralheizung,
            feature_abstellraum,
            feature_fernwaerme,
            feature_fussbodenheitzung,
            feature_gartenmitbenutzung,
            feature_kellerabteil,
            erklärung
        ],
        #gebe mir den output der funktion in einem number gradio widget aus
        outputs=[output_html, waterfall_plot]
    )

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


xgb loaded
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=30,
             enable_categorical=False, eval_metric=['rmse', 'mae'],
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=42, ...)
Data cols: Index(['LivingSpace', 'Rooms', 'altbau_(bis_1945)', 'balkon', 'barriefrei',
       'dachgeschoss', 'einbaukueche', 'neubau', 'parkett', 'stellplatz',
       'bad/wc_getrennt', 'personenaufzug', 'garten', 'garage', 'renoviert',
     

Traceback (most recent call last):
  File "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\shap\explainers\_tree.py", line 358, in shap_values
    phi = self.model.original_model.predict(
  File "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\xgboost\core.py", line 2163, in predict
    _check_call(
  File "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\xgboost\core.py", line 279, in _check_call
    raise XGBoostError(py_str(_LIB.XGBGetLastError()))
xgboost.core.XGBoostError: [23:53:07] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\c_api\c_api_utils.h:121: Check failed: std::accumulate(shape.cbegin(), shape.cend(), static_cast<bst_ulong>(1), std::multiplies<>{}) == chunksize * rows (2 vs. 47) : 

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\gra